In [36]:
# This notebook is used for testing.
# Aim: Calculate magnetic fields of different coil configuration

In [ ]:
#Version1:
#Includes formula for calculating the B-field of a straight line from r_start to r_end at any arbitrary position
#Includes formula to calculate B-field of many straight lines; line segments are specified using a tuple_list
#Includes formula to create tuple_list for a single loop

In [53]:
import numpy as np
import scipy.constants
import timeit
import transforms3d #To get rotation matrices
import math

In [38]:
#Define constants:
mu_0=scipy.constants.mu_0
pi=scipy.constants.pi

In [39]:
#Coordinates where we want to calculate the magnetic field:
x_0=0
y_0=0
z_0=0
r_0=np.array([x_0,y_0,z_0])
#print(r_0)
r_start=np.array([1,1,0])
r_end=np.array([-1,1,0])

[0 0 0]


In [40]:
#Define some nice functions:
#Function to normalize a vector:
def normalize(a):
    norm = np.linalg.norm(a)
    if norm==0:
        norm = 1
    return a / norm

#Function to get the sign of the angles of two vectors (e_r_01 and e_r_02) with respect to a third one (e_R):
#angles 1 and 2 define the mathematically positive direction
def angle_sign(def_vec1, def_vec2, ref_vec, vec_of_interest):
    x=np.dot(np.cross(def_vec1,def_vec2),np.cross(ref_vec,vec_of_interest))
    if x>0:
        return 1
    else:
        return -1
    

In [41]:
# Function to calculate magnetic field created by a straight piece of wire:
def B_straight_line(r_start,r_end,r_0,I):
    '''r_0 is the point where the B-field is calculated, the wire starts at r_start and ends at r_end, I is the current from r-start to r_end'''
    #Define vectors and corresponding unit vectors:
    r_01=r_start-r_0
    e_r_01=normalize(r_01)
    r_02=r_end-r_0
    e_r_02=normalize(r_02)
    r_wire=r_end-r_start
    e_r_wire=normalize(r_wire)
    #R is the vector from r_0 which is perpendicular to r_wire
    e_R=np.cross(r_wire,np.cross(r_01,r_wire))/(np.linalg.norm(r_wire)*np.linalg.norm(np.cross(r_01,r_wire)))
    R=np.linalg.norm(np.dot(r_01,e_R))*e_R
    #checked: R is perpendicular, R has correct length and is the correct vector
    #print(np.dot(R,r_wire)) #R perpendicular to wire if =0
    #print(np.linalg.norm(R))
    #print(R)
    
    
    #Find angles of r_start and r_end with respect to e_R:
    #Problem: By just taking the vector product or the dot product you get positive angles. So the plan is to use the dot product between e_r_01 and e_r_02 to define a right-handed system. If the dot-product of e_R with e_r_01/e_r_02 is in the same direction, their angle is positive with respect to e_R. -> See Function angle_sign(). 
    alpha_1=angle_sign(e_r_01,e_r_02,e_R,e_r_01)*np.arccos((np.linalg.norm(np.cross(r_start,r_wire)))/(np.linalg.norm(r_start)*np.linalg.norm(r_wire)))
    alpha_2=angle_sign(e_r_01,e_r_02,e_R,e_r_02)*np.arccos((np.linalg.norm(np.cross(r_end,r_wire)))/(np.linalg.norm(r_end)*np.linalg.norm(r_wire)))
    #Check angles:
    #print(alpha_1*360/(2*pi),alpha_2*360/(2*pi))
    
    #Get the unit vector of the B-field
    e_t=np.cross(e_R,e_r_wire)
    #print(e_t)
    
    return mu_0*I/(4*pi*np.linalg.norm(R))*(np.sin(alpha_2)-np.sin(alpha_1))*e_t

In [42]:
#Check formula B_straight_line:
# Use a very long wire along the x-axis and get the B-field at arbitrary points in space
#Coordinates where we want to calculate the magnetic field:
x_0=0
y_0=10
z_0=0
r_0=np.array([x_0,y_0,z_0])
#print(r_0)
r_start=np.array([-10000,0,0])
r_end=np.array([10000,0,0])
result=B_straight_line(r_start,r_end,r_0,1)
#print(result)
#print(mu_0*1/(2*pi*10))
#Direction of field and magnitude are correct

#To check the formula, using a square loop with edge length a=0.1m, current I=1A and the B-field is measured at the symmetry axis in the plane of the loop
#Wikipedia, Biot-Savart says for a square loop on the symmetry axis and in the plane of the loop:
def square_loop_wikicheck(a,I):
    return (mu_0*I*2/pi)*np.sqrt(2/(a**2))
#print(square_loop_wikicheck(0.2,1))
#My straight-line-function
x_0=0
y_0=0
z_0=0
r_0=np.array([x_0,y_0,z_0])
square_loop=B_straight_line(np.array([0.1,-0.1,-0.1]),np.array([0.1,0.1,-0.1]),r_0,1)+B_straight_line(np.array([0.1,0.1,-0.1]),np.array([0.1,0.1,0.1]),r_0,1)+B_straight_line(np.array([0.1,0.1,0.1]),np.array([0.1,-0.1,0.1]),r_0,1)+B_straight_line(np.array([0.1,-0.1,0.1]),np.array([0.1,-0.1,-0.1]),r_0,1)
#print(square_loop)
#Test passed

[  2.30940108e-06   0.00000000e+00   0.00000000e+00]


In [43]:
#Formula for a circular loop with radius R, B-field measured on axis with distance z to the plane of the loop
def B_circular_loop_onaxis(z,R,I):
    return mu_0*I*R**2/(2*np.power(R**2+z**2,1.5))

#print(B_circular_loop_onaxis(0.1,0.1,1))

2.22144146908e-06


In [47]:
#Make function work with tuples [r_start,r_end]
def B_field(vector_list,r_0,I):
#Map the vector tuples onto the vector arguments of B_straight line (r_start,r_end) and add all resulting B-field vectors together
    return np.sum((map(lambda (r_start,r_end): B_straight_line(r_start,r_end,r_0,I), vector_list)), axis=0)


#my_list=[(np.array([0.1,-0.1,-0.1]),np.array([0.1,0.1,-0.1])),(np.array([0.1,-0.1,-0.1]),np.array([0.1,0.1,-0.1])),(np.array([0.1,-0.1,-0.1]),np.array([0.1,0.1,-0.1])),(np.array([0.1,-0.1,-0.1]),np.array([0.1,0.1,-0.1]))]

#print(B_field(my_list,r_0,1))



[  2.30940108e-06   0.00000000e+00   2.30940108e-06]


In [104]:
#Create a function that creates list of vector tuples of a regular polygon loop:

# Function to create rotation matrix:
def rotation_matrix(axis, theta):
    """
    Return the rotation matrix associated with counterclockwise rotation about
    the given axis (np.array) by theta radians. See Euler–Rodrigues formula (Wikipedia)
    """
    axis = normalize(axis)
    a = np.cos(theta/2.0)
    b, c, d = -axis*np.sin(theta/2.0)
    aa, bb, cc, dd = a*a, b*b, c*c, d*d
    bc, ad, ac, ab, bd, cd = b*c, a*d, a*c, a*b, b*d, c*d
    return np.array([[aa+bb-cc-dd, 2*(bc+ad), 2*(bd-ac)],
                     [2*(bc-ad), aa+cc-bb-dd, 2*(cd+ab)],
                     [2*(bd+ac), 2*(cd-ab), aa+dd-bb-cc]])

def theta(N):
    return 2*pi/N

def tuple_list_single_loop(N,e_normal,e_start_side,r_center,r_in):
    #ToDo include normalization of all e_... input vectors
    #include a test that e_start and e_normal are perpendicular
    r_out=r_in/(np.cos(pi/N))
    e_start_corner=np.dot(rotation_matrix(e_normal,theta(N)/2),e_start_side) 
    r_start_corner=r_out*e_start_corner
    
    
    
    #Create list of vector tuples:
    #Use the starting edge vector and rotate it around the symmetry axis by 2pi/N to get the next corner. Every segment goes from one to the next corner.
    tuple_list=[(r_center+np.dot(rotation_matrix(e_normal,i*theta(N)),r_start_corner),r_center+np.dot(rotation_matrix(e_normal,(i+1)*theta(N)),r_start_corner)) for i in range(N)]
    return tuple_list

#corner_vector_list=[np.dot(rotation_matrix(e_normal,i*theta),r_start_corner) for i in range(N)]
#print(corner_vector_list)
e_normal=np.array([1,0,0])/np.linalg.norm(np.array([1,0,]))
e_start_side=np.array([-1,1,1])/np.linalg.norm(np.array([-1,1,1]))
r_center=np.array([0,0,0])
N=4
r_in=1
print(tuple_list_single_loop(4,e_normal,e_start_side,r_center,r_in))

[(array([-0.98559856, -0.23914631,  0.98559856]), array([ 0.16910198, -1.39384685, -0.16910198])), (array([ 0.16910198, -1.39384685, -0.16910198]), array([ 0.98559856,  0.23914631, -0.98559856])), (array([ 0.98559856,  0.23914631, -0.98559856]), array([-0.16910198,  1.39384685,  0.16910198])), (array([-0.16910198,  1.39384685,  0.16910198]), array([-0.98559856, -0.23914631,  0.98559856]))]
